In [2]:
import sunpy
from datetime import datetime
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sunpy.net import Fido, attrs as a
from sunpy.time import TimeRange

/Users/hippolytehilgers/Desktop/ROSACE/Project_ROSACE/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
start_time = datetime(2022, 1, 1)
end_time   = datetime(2022, 2, 28)

query = Fido.search(
    a.Time(start_time, end_time),
    a.Instrument.lasco,
    a.Detector.c2,
)

In [16]:
# Show the results
print(query)

Results from 1 Provider:

7276 Results from the VSOClient:
Source: https://sdac.virtualsolar.org/cgi/search
Data retrieval status: https://docs.virtualsolar.org/wiki/VSOHealthReport
Total estimated size: 14.082 Gbyte

       Start Time               End Time        Source Instrument Provider  Physobs  Extent Type   Size 
                                                                                                  Mibyte
----------------------- ----------------------- ------ ---------- -------- --------- ----------- -------
2022-01-01 02:33:04.000 2022-01-01 02:33:29.000   SOHO      LASCO     SDAC intensity      CORONA 2.01074
2022-01-01 02:48:05.000 2022-01-01 02:48:30.000   SOHO      LASCO     SDAC intensity      CORONA 2.01074
2022-01-01 02:54:09.000 2022-01-01 02:55:50.000   SOHO      LASCO     SDAC intensity      CORONA 1.01074
2022-01-01 02:58:01.000 2022-01-01 02:59:41.000   SOHO      LASCO     SDAC intensity      CORONA 1.01074
2022-01-01 03:01:52.000 2022-01-01 03:03:32.000

In [5]:
# Download just the first file
downloaded_files = Fido.fetch(query[0, 0])
print(f"Downloaded file: {downloaded_files[0]}")

Files Downloaded: 100%|██████████| 1/1 [00:01<00:00,  1.79s/file]

Downloaded file: /Users/hippolytehilgers/sunpy/data/22842625.fts


In [25]:
fts_to_png(
    input_path="/Users/hippolytehilgers/sunpy/data/22842625.fts",
    output_dir="WP2/data_lasco_c2/data_lasco_2022_png"
)

✅ Sauvé : WP2/data_lasco_c2/data_lasco_2022_png/LASCOC2_2022_01_01_02_33_04_805.png


'WP2/data_lasco_c2/data_lasco_2022_png/LASCOC2_2022_01_01_02_33_04_805.png'

In [24]:
from PIL import Image
import numpy as np
import sunpy.map
import os
from datetime import datetime
import re

def fts_to_png(input_path, output_dir, size=(512, 512), contrast_clip=(1, 99)):
    """
    Convertit un fichier LASCO .fts/.fits en PNG 512x512.
    Le nom du PNG est basé sur la combinaison DATE-OBS + TIME-OBS.
    """

    try:
        # Lecture du FITS via SunPy
        smap = sunpy.map.Map(input_path)
        data = smap.data.astype(float)
        header = smap.meta

        # Lecture des champs DATE-OBS et TIME-OBS
        date_part = header.get('DATE-OBS', '')
        time_part = header.get('TIME-OBS', '')

        if date_part and time_part:
            date_str_full = f"{date_part}T{time_part}"
        elif date_part:  # parfois TIME-OBS manquant, fallback
            date_str_full = date_part
        else:
            # fallback : nom du fichier
            date_str_full = os.path.splitext(os.path.basename(input_path))[0]

        # Conversion en format standard YYYYMMDD_HHMMSS
        try:
            date_obj = datetime.strptime(date_str_full, "%Y-%m-%dT%H:%M:%S")
            date_str = date_obj.strftime("%Y%m%d_%H%M%S")
        except Exception:
            date_str = re.sub(r'[^0-9]', '_', date_str_full)

        # Nettoyage des caractères spéciaux (sécurité)
        date_str = re.sub(r'[^0-9A-Za-z_]', '_', date_str)

        # Normalisation de l'image
        data = np.nan_to_num(data, nan=0.0, posinf=0.0, neginf=0.0)
        vmin, vmax = np.percentile(data, contrast_clip)
        data = np.clip(data, vmin, vmax)
        data = (255 * (data - vmin) / (vmax - vmin)).astype(np.uint8)

        # Conversion en PNG 512x512
        img = Image.fromarray(data)
        img = img.resize(size, Image.Resampling.LANCZOS)

        # Nom de sortie avec la date/heure incluse
        os.makedirs(output_dir, exist_ok=True)
        filename = f"LASCOC2_{date_str}.png"
        output_path = os.path.join(output_dir, filename)

        img.save(output_path)
        print(f"✅ Sauvé : {output_path}")

        return output_path

    except Exception as e:
        print(f"❌ Erreur pour {input_path} : {e}")
        return None
